# 1. Candidate generation

In [2]:
import pandas as pd 
import numpy as np
import random


***Read Files***

In [3]:
#df = pd.read_csv('data/user_videos_updated.csv')
# todo: config lại để chỉ drop na tại những cột mình xài
df = pd.read_csv('data/user_videos_updated.csv')
df = df.dropna()
df.count()


user_id               1476
user_gender           1476
user_zip              1476
dwell_time            1476
watchedAt             1476
                      ... 
discussion_density    1476
like_percentage       1476
dislike_percentage    1476
like_dislike_ratio    1476
share_percentage      1476
Length: 120, dtype: int64

In [4]:
# map user id with index
userMap = {v:k for k,v in enumerate(df['user_id'].unique())}
df['user_id'] = df['user_id'].map(userMap)
# map video id with index
videoMap = {v:k for k,v in enumerate(df['video_id'].unique())}
df['video_id'] = df['video_id'].map(videoMap)

In [5]:
t = df.groupby('video_id')['user_id'].count()
t.describe()


count    1435.000000
mean        1.028571
std         0.204258
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         4.000000
Name: user_id, dtype: float64

In [6]:
# calculate example age
df['example_age']=(pd.to_datetime("now")-pd.to_datetime(df['publishedAt']).dt.tz_localize(None))/ np.timedelta64(1,'D')

In [7]:
def normalize_col(df,col_name):
    df[col_name] = (df[col_name] - df[col_name].mean()) / (df[col_name].std() + 1e-8)
    return df

In [8]:
df['like/dislike'] = [True if x >= 160 else False for _,x in enumerate(df['dwell_time'].tolist())]
df = normalize_col(df,'example_age')
df = normalize_col(df,'dwell_time')
df = normalize_col(df,'watchTime')


In [9]:
df['example_age']

0      -0.547329
1       0.306015
2      -0.749816
3      -0.805356
4       0.081615
          ...   
1807   -0.626633
1808    1.486589
1810    0.016956
1811   -0.894626
1813   -0.154344
Name: example_age, Length: 1476, dtype: float64

In [10]:
df['like/dislike'].value_counts()

False    773
True     703
Name: like/dislike, dtype: int64

In [12]:
users = df.iloc[:,:3]
videos = df.iloc[:,7:]
videos.count()


user_id        1476
user_gender    1476
user_zip       1476
dtype: int64

In [164]:
videos['year'] = videos['publishedAt'].apply(lambda x:str(x).split('-')[0])


In [165]:
watch_cols = ['user_id', 'video_id','dwell_time','like/dislike','watchedAt']
watch = df[watch_cols]
watch['year'] = watch['watchedAt'].apply(lambda x:str(x).split('-')[0])
watch = normalize_col(watch,'dwell_time')

C:\Users\I7\AppData\Local\Temp/ipykernel_25656/4189475514.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  watch['year'] = watch['watchedAt'].apply(lambda x:str(x).split('-')[0])
C:\Users\I7\AppData\Local\Temp/ipykernel_25656/1005285834.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = (df[col_name] - df[col_name].mean()) / (df[col_name].std() + 1e-8)


In [166]:
# like & dislike both false -> like
# like & dislike both true -> random false
watch['like/dislike'] = df['like/dislike']
watch['like/dislike'].unique()

C:\Users\I7\AppData\Local\Temp/ipykernel_25656/324578433.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  watch['like/dislike'] = df['like/dislike']


array([False,  True])

In [167]:
# genres encoding
genres = df['category_term'].unique().tolist()
genres_encoded = {x: i for i,x in enumerate(genres)}
genres_encoded_2_genres = {i:x for i,x in enumerate(genres)}


# user encoding
user_ids = users['user_id'].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}

# movie index encoding
video_ids = videos["video_id"].unique().tolist()
video2video_encoded = {x: i for i, x in enumerate(video_ids)}
video_encoded2video = {i: x for i, x in enumerate(video_ids)}

# movie title encoding
title_ids = videos['video_title'].unique().tolist()
#user_video_fond = like_dislike.pivot(index='user', columns='like/dislike', values='video').reset_index()
#user_video_fond.loc[user_video_fond['like'].isnull(),['like']] = user_video_fond.loc[user_video_fond['like'].isnull(),'like'].apply(lambda x: [dfenc['video'].max()+1] ) = {x: i for i, x in enumerate(title_ids)}
title_encoded2title = {i: x for i, x in enumerate(title_ids)}

#video category encoding 
cats = videos['category_term'].unique().tolist()
cat2cat_encoded = {x: i for i,x in enumerate(cats)}
cat_encoded2cat = {x:i for x,i in enumerate(cats)}

# genders encoding
gender_ids = users['user_gender'].unique().tolist()
gender2gender_encoded = {x: i for i, x in enumerate(gender_ids)}
gender_encoded2gender = {i: x for i, x in enumerate(gender_ids)}

#like/disklike encoding
like_dislike_ids =  watch['like/dislike'].unique().tolist()
like_dislike_ids2encoded = {x: i for i, x in enumerate(like_dislike_ids)}
encoded2like = {i: x for i, x in enumerate(like_dislike_ids)}




In [168]:
# # Only do this one time
likeMap = {True:'like', False:'dislike'}
df['like/dislike'] = df['like/dislike'].map(likeMap)

In [169]:
df['like/dislike']

0       dislike
1          like
2          like
3       dislike
4          like
         ...   
1807       like
1808    dislike
1810       like
1811       like
1813    dislike
Name: like/dislike, Length: 1476, dtype: object

In [170]:

dfenc = df
# Change to encoding
title2title_encoded = {x: i for i, x in enumerate(title_ids)}
dfenc["title_id"] = df["video_title"].map(title2title_encoded)

dfenc["user"] = df["user_id"].map(user2user_encoded)
dfenc["video"] = df["video_id"].map(video2video_encoded)
dfenc["title_d"] = df["video_title"].map(title2title_encoded)
dfenc['gender_d'] = df['user_gender'].map(gender2gender_encoded)
dfenc['cat_d'] = df['category_term'].map(cat2cat_encoded)

watch_hist = dfenc.groupby(['user'])['video_id'].apply(list).reset_index()

watch_hist_time = dfenc.groupby(['user'])['dwell_time'].apply(list).reset_index()

like_dislike = dfenc.groupby(['user','like/dislike'])['video'].apply(list).reset_index()

example_age = dfenc.groupby(['user'])['example_age'].apply(list).reset_index()


user_video_fond = like_dislike.pivot(index='user', columns='like/dislike', values='video').reset_index()
user_video_fond.fillna(dfenc["video"].max()+1, inplace=True)
user_video_fond['dislike'] = user_video_fond['dislike'].apply(lambda x: (x)) 
user_video_fond['like'] = user_video_fond['like'].apply(lambda x: (x))


# coi lại chỗ này -> 1435 (max) đang nhiều
user_video_list = dfenc.pivot(index='user_id', columns='video_id', values='video').reset_index()
user_video_fond = like_dislike.pivot(index='user', columns='like/dislike', values='video').reset_index()
user_video_fond.loc[user_video_fond['like'].isnull(),['like']] = user_video_fond.loc[user_video_fond['like'].isnull(),'like'].apply(lambda x: [dfenc['video'].max()+1] )

sample_data=dfenc[['user','user_gender']]
# sample_data=sample_data.reset_index()
sample_data = sample_data.drop_duplicates()

user_movie_list = pd.merge(sample_data,watch_hist, how= 'left')
user_movie_list = pd.merge(user_movie_list,watch_hist_time, how='left')
user_movie_list = pd.merge(user_movie_list,example_age, how='left')
user_movie_list = pd.merge(user_movie_list,user_video_fond,how='left')

# generate predict labels
user_movie_list['predict_labels'] = user_movie_list['video_id'].apply(lambda x: int(random.uniform(0,dfenc["video"].max()))) #label을 마지막 값으로..


# split 60:40
# train_size = user_movie_list.user.max() * 0.6
# # train_size = 10 * 0.6
# test_size = user_movie_list.user.max() - train_size
# train_data = user_movie_list[(user_movie_list.user >= 1)&
#                                   (user_movie_list.user <= tr)]
# test_data = user_movie_list[(user_movie_list.user >= train_size+1)&
#                                   (user_movie_list.user <= 10)]

In [171]:
# user_movie_list['predict_labels'] = user_movie_list['like'].apply(lambda x: (x[-1]) )
# user_movie_list['like'] = user_movie_list['like'].apply(lambda x: x[:-1])

In [172]:
# split test and train set
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(user_movie_list, test_size=0.2)


In [173]:
train_data

,user,user_gender,video_id,dwell_time,example_age,dislike,like,predict_labels
117,117,Female,"[1066, 1067, 1068, 1069]","[-0.23120339815695018, -0.655470440851881, -0....","[-0.752789134354159, -0.7572088549577456, -0.8...","[1066, 1067, 1068, 1069]",[1435],657
80,80,Male,"[603, 604, 605, 606]","[-0.6153475865599131, -0.3508333917128816, -0....","[0.2669779600197808, 1.3510923831282715, 0.227...","[603, 604, 605]",[606],1206
141,141,Female,"[1361, 957, 1362, 1363, 1364, 1365, 1366, 1367...","[-0.1669904186382468, 0.11500932887461997, -0....","[0.34228584529790385, 0.40544969958947885, 2.1...","[1362, 1363, 1369, 1370, 1371, 1372, 1373, 137...","[1361, 957, 1364, 1365, 1366, 1367, 1368, 1374...",1233
136,136,Female,"[1271, 1272, 1273, 1274, 1275, 1276, 1277, 876...","[0.695682862128195, -0.6670930296567051, 0.866...","[0.780275751993145, -0.1737072983406337, 0.408...",[1272],"[1271, 1273, 1274, 1275, 1276, 1277, 876, 1278...",967
145,145,Female,"[1418, 1419, 1420, 1421, 214, 1422, 39, 1423, ...","[-0.009772413641608878, -0.6806669003686154, 0...","[-0.8572521146743189, -0.9136670678485957, 1.7...","[1419, 1421, 214, 1422, 1423, 1425]","[1418, 1420, 39, 1424]",664
...,...,...,...,...,...,...,...,...
144,144,Female,[219],[-0.16083871368506575],[-0.7801721145240683],NaN,[219],488
18,18,Female,"[72, 73, 74]","[-0.12787055971908023, -0.3902237554447127, 0....","[1.4700045859636774, -0.76830454588044, -0.703...",[73],"[72, 74]",429
104,104,Female,"[870, 871, 872, 873, 874, 875, 876]","[-0.6715122880568658, -0.1335785152042947, 0.0...","[-0.7612984949810746, -0.0036184870481447322, ...","[870, 874, 875, 876]","[871, 872, 873]",397
107,107,Male,"[887, 888, 889, 890, 891, 892, 893, 894, 895, ...","[-0.5296796242030157, -0.6264869139578423, -0....","[0.759839260785851, 3.028590146878647, -0.5082...","[887, 888, 890, 891, 892, 893, 894, 897]","[889, 895, 896]",46


In [174]:
user_movie_list

,user,user_gender,video_id,dwell_time,example_age,dislike,like,predict_labels
0,0,Female,"[0, 1]","[-0.38683180745669393, 1.652215234129411]","[-0.5473293143141689, 0.30601517821658614]",[0],[1],740
1,1,Male,"[2, 3, 4]","[0.11961398944975296, -0.6393313334895335, -0....","[-0.7498156960918726, -0.805355604630498, 0.08...",[3],"[2, 4]",919
2,2,Male,[5],[-0.12214436795436039],[0.8014712518304528],NaN,[5],389
3,3,Female,"[6, 7, 8]","[-0.6794602665413363, 0.25142885709526097, 0.2...","[2.2943273329368004, 1.6946288562920628, 1.695...",[6],"[7, 8]",722
4,4,Female,[9],[-0.534658128508011],[2.0662103270767838],[9],[1435],353
...,...,...,...,...,...,...,...,...
143,143,Female,"[1411, 1412, 1413, 1414, 1415, 1416, 1417]","[0.7568929342883295, -0.6447110359436492, 0.17...","[-6.120972349872735e-05, 1.7882759279366218, -...",[1412],"[1411, 1413, 1414, 1415, 1416, 1417]",298
144,144,Female,[219],[-0.16083871368506575],[-0.7801721145240683],NaN,[219],488
145,145,Female,"[1418, 1419, 1420, 1421, 214, 1422, 39, 1423, ...","[-0.009772413641608878, -0.6806669003686154, 0...","[-0.8572521146743189, -0.9136670678485957, 1.7...","[1419, 1421, 214, 1422, 1423, 1425]","[1418, 1420, 39, 1424]",664
146,146,Female,"[1426, 1427, 1428, 1429, 1430, 1431, 1432, 1433]","[0.12305152813653554, -0.45124538604995756, 0....","[-0.3761021150192883, -0.3609177122185179, -0....","[1427, 1431]","[1426, 1428, 1429, 1430, 1432, 1433]",647


### Hyperparameter Definition

In [175]:
# CONTINUE FROM HERE - 30-09-2021
EMBEDDING_DIMS = 16
DENSE_UNITS = 64
DROPOUT_PCT = 0.0
ALPHA = 0.0
NUM_CLASSES= df["video_id"].max() + 2
LEARNING_RATE = 0.001

In [176]:

import tensorflow as tf
tf.random.set_seed(
    1024
)
class MaskedEmbeddingsAggregatorLayer(tf.keras.layers.Layer):
    def __init__(self, agg_mode='sum', **kwargs):
        super(MaskedEmbeddingsAggregatorLayer, self).__init__(**kwargs)

        if agg_mode not in ['sum', 'mean']:
            raise NotImplementedError('mode {} not implemented!'.format(agg_mode))
        self.agg_mode = agg_mode
    
    @tf.function
    def call(self, inputs, mask=None):
#         https://www.tensorflow.org/api_docs/python/tf/ragged/boolean_mask
        masked_embeddings = tf.ragged.boolean_mask(inputs, mask)
        if self.agg_mode == 'sum':
            aggregated =  tf.reduce_sum(masked_embeddings, axis=1)
        elif self.agg_mode == 'mean':
            aggregated = tf.reduce_mean(masked_embeddings, axis=1)
        return aggregated
    
    def get_config(self):
        # this is used when loading a saved model that uses a custom layer
        return {'agg_mode': self.agg_mode}
    
class L2NormLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(L2NormLayer, self).__init__(**kwargs)
    
    @tf.function
    def call(self, inputs, mask=None):
        if mask is not None:
            inputs = tf.ragged.boolean_mask(inputs, mask).to_tensor()
        return tf.math.l2_normalize(inputs, axis=-1)

    def compute_mask(self, inputs, mask):
        return mask

In [177]:
#---inputs
import tensorflow as tf
import datetime
import os
input_video_id = tf.keras.Input(shape=(None, ), name='video_id')
input_dwell_time = tf.keras.layers.Input(shape=(None,), name='dwell_time')
input_like = tf.keras.layers.Input(shape=(None,), name='like')
input_example_age = tf.keras.Input(shape=(None, ), name='example_age')
# input_occupation = tf.keras.Input(shape=(None, ), name='occupation')


#--- layers
features_embedding_layer = tf.keras.layers.Embedding(input_dim=NUM_CLASSES, output_dim=EMBEDDING_DIMS, 
                                            mask_zero=True, trainable=True, name='features_embeddings')
labels_embedding_layer = tf.keras.layers.Embedding(input_dim=NUM_CLASSES, output_dim=EMBEDDING_DIMS, 
                                            mask_zero=True, trainable=True, name='labels_embeddings')

avg_embeddings = MaskedEmbeddingsAggregatorLayer(agg_mode='mean', name='aggregate_embeddings')

dense_1 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_1')
dense_2 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_2')
dense_3 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_3')
l2_norm_1 = L2NormLayer(name='l2_norm_1')

dense_output = tf.keras.layers.Dense(NUM_CLASSES, activation=tf.nn.softmax, name='dense_output')

#--- features
features_embeddings = features_embedding_layer(input_video_id)
l2_norm_features = l2_norm_1(features_embeddings)
avg_features = avg_embeddings(l2_norm_features)

labels_watch_embeddings = labels_embedding_layer(input_dwell_time)
l2_norm_watched = l2_norm_1(labels_watch_embeddings)
avg_watched = avg_embeddings(l2_norm_watched)

labels_like_embeddings = labels_embedding_layer(input_like)
l2_norm_like = l2_norm_1(labels_like_embeddings)
avg_liked = avg_embeddings(l2_norm_like)

labels_example_age_embeddings = labels_embedding_layer(input_example_age)
l2_norm_example_age = l2_norm_1(labels_example_age_embeddings)
avg_example_age = avg_embeddings(l2_norm_example_age)

# labels_occupation_embeddings = labels_embedding_layer(input_occupation)
# l2_norm_occupation = l2_norm_1(labels_occupation_embeddings)
# avg__occupation = avg_embeddings(l2_norm_occupation)


print(avg_features)
print(avg_watched)
print(avg_liked)
print(avg_example_age)
# print(input_occupation)

# 임베딩 벡터들 연결
concat_inputs = tf.keras.layers.Concatenate(axis=1)([avg_features,
                                                     avg_watched,
                                                     avg_liked,
                                                     avg_example_age,
#                                                      avg__occupation
                                                     ])
# Dense Layers
dense_1_features = dense_1(concat_inputs)
dense_1_relu = tf.keras.layers.ReLU(name='dense_1_relu')(dense_1_features)
dense_1_batch_norm = tf.keras.layers.BatchNormalization(name='dense_1_batch_norm')(dense_1_relu)

dense_2_features = dense_2(dense_1_relu)
dense_2_relu = tf.keras.layers.ReLU(name='dense_2_relu')(dense_2_features)
# dense_2_batch_norm = tf.keras.layers.BatchNormalization(name='dense_2_batch_norm')(dense_2_relu)

dense_3_features = dense_3(dense_2_relu)
dense_3_relu = tf.keras.layers.ReLU(name='dense_3_relu')(dense_3_features)
dense_3_batch_norm = tf.keras.layers.BatchNormalization(name='dense_3_batch_norm')(dense_3_relu)
# for ranking we use this line
# outputs = dense_output(dense_3_batch_norm) + tf.keras.layers.Softmax(dense_3_batch_norm)
# for canndidate generation
outputs = dense_output(dense_3_batch_norm)


#Optimizer
optimiser = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

#--- prep model
model = tf.keras.models.Model(
    inputs=[input_video_id, 
            input_dwell_time, 
            input_like,
            input_example_age,
            ],
    outputs=[outputs]
)
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
et = tf.keras.callbacks.EarlyStopping(
    monitor="acc",
    #min_delta=0,
    patience=50,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)


# model.compile(optimizer=optimiser, loss = lambda y_pred,y_true: tf.keras.losses.CategoricalCrossentropy()(tf.keras.layers.Softmax(y_pred), y) )
model.compile(optimizer=optimiser, loss='sparse_categorical_crossentropy', metrics=['acc'])

model.summary()

KerasTensor(type_spec=TensorSpec(shape=(None, 16), dtype=tf.float32, name=None), name='aggregate_embeddings/PartitionedCall:0', description="created by layer 'aggregate_embeddings'")
KerasTensor(type_spec=TensorSpec(shape=(None, 16), dtype=tf.float32, name=None), name='aggregate_embeddings/PartitionedCall:0', description="created by layer 'aggregate_embeddings'")
KerasTensor(type_spec=TensorSpec(shape=(None, 16), dtype=tf.float32, name=None), name='aggregate_embeddings/PartitionedCall:0', description="created by layer 'aggregate_embeddings'")
KerasTensor(type_spec=TensorSpec(shape=(None, 16), dtype=tf.float32, name=None), name='aggregate_embeddings/PartitionedCall:0', description="created by layer 'aggregate_embeddings'")
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
watch_hist (InputLayer)         [(None, None)]       

In [178]:
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True,dpi=96)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [179]:
history = model.fit([tf.keras.preprocessing.sequence.pad_sequences(train_data['video_id']),
           tf.keras.preprocessing.sequence.pad_sequences(train_data['dwell_time'], dtype=float),
           tf.keras.preprocessing.sequence.pad_sequences(train_data['like']) + 1e-10,
           tf.keras.preprocessing.sequence.pad_sequences(train_data['example_age'], dtype=float),
#            tf.keras.preprocessing.sequence.pad_sequences(train_data['occupation'], dtype=float),
           ],train_data['predict_labels'].values,
           steps_per_epoch=1, epochs=5000,callbacks=[et])




Epoch 1/5000
1/1 [==============================] - 1s 1s/step - loss: 7.2617 - acc: 0.0000e+00
Epoch 2/5000
1/1 [==============================] - 0s 7ms/step - loss: 7.2325 - acc: 0.0085
Epoch 3/5000
1/1 [==============================] - 0s 9ms/step - loss: 7.2067 - acc: 0.0254
Epoch 4/5000
1/1 [==============================] - 0s 7ms/step - loss: 7.1799 - acc: 0.0254
Epoch 5/5000
1/1 [==============================] - 0s 8ms/step - loss: 7.1525 - acc: 0.0339
Epoch 6/5000
1/1 [==============================] - 0s 9ms/step - loss: 7.1238 - acc: 0.0508
Epoch 7/5000
1/1 [==============================] - 0s 8ms/step - loss: 7.0938 - acc: 0.1102
Epoch 8/5000
1/1 [==============================] - 0s 10ms/step - loss: 7.0637 - acc: 0.1271
Epoch 9/5000
1/1 [==============================] - 0s 7ms/step - loss: 7.0317 - acc: 0.1356
Epoch 10/5000
1/1 [==============================] - 0s 8ms/step - loss: 6.9983 - acc: 0.1356
Epoch 11/5000
1/1 [==============================] - 0s 10ms/step

1/1 [==============================] - 0s 8ms/step - loss: 3.2122 - acc: 0.4492
Epoch 89/5000
1/1 [==============================] - 0s 8ms/step - loss: 3.1745 - acc: 0.4492
Epoch 90/5000
1/1 [==============================] - 0s 8ms/step - loss: 3.1372 - acc: 0.4576
Epoch 91/5000
1/1 [==============================] - 0s 8ms/step - loss: 3.1002 - acc: 0.4576
Epoch 92/5000
1/1 [==============================] - 0s 7ms/step - loss: 3.0641 - acc: 0.4661
Epoch 93/5000
1/1 [==============================] - 0s 9ms/step - loss: 3.0286 - acc: 0.4746
Epoch 94/5000
1/1 [==============================] - 0s 7ms/step - loss: 2.9939 - acc: 0.4746
Epoch 95/5000
1/1 [==============================] - 0s 8ms/step - loss: 2.9601 - acc: 0.4746
Epoch 96/5000
1/1 [==============================] - 0s 8ms/step - loss: 2.9246 - acc: 0.4746
Epoch 97/5000
1/1 [==============================] - 0s 7ms/step - loss: 2.8914 - acc: 0.4746
Epoch 98/5000
1/1 [==============================] - 0s 8ms/step - loss: 2

In [180]:
model.save("candidate_generation.h5")

D:\conda\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [181]:
pred = model.predict([tf.keras.preprocessing.sequence.pad_sequences(test_data['video_id']),
           tf.keras.preprocessing.sequence.pad_sequences(test_data['dwell_time'], dtype=float),
           tf.keras.preprocessing.sequence.pad_sequences(test_data['like']),
           tf.keras.preprocessing.sequence.pad_sequences(test_data['example_age'], dtype=float)
           ])

In [182]:
pred

array([[0.00047801, 0.00047948, 0.00049305, ..., 0.00046682, 0.00040656,
        0.00041262],
       [0.00047801, 0.00047948, 0.00049305, ..., 0.00046682, 0.00040656,
        0.00041262],
       [0.00052215, 0.00051591, 0.000537  , ..., 0.00051047, 0.00045144,
        0.00045265],
       ...,
       [0.00047801, 0.00047948, 0.00049305, ..., 0.00046682, 0.00040656,
        0.00041262],
       [0.00047801, 0.00047948, 0.00049305, ..., 0.00046682, 0.00040656,
        0.00041262],
       [0.00058045, 0.00061742, 0.00061595, ..., 0.00057963, 0.00055311,
        0.00051996]], dtype=float32)

In [183]:
# candidate generation: 
###### We extract the top-7 recommendation data for each user.

#???????????????????????????????????????????????????????????????????
N = 6
k = np.sort((-pred).argsort()[:,:N])
print(k)
k = k.flatten()
k[k>df["video"].max()]=0
k = np.unique(k)


[[  91  206  456  587  649 1137]
 [  91  206  456  587  649 1137]
 [  91  206  456  587  649 1137]
 [ 189  226  587 1223 1233 1371]
 [  91  206  456  587  649 1137]
 [  91  206  456  587  649 1137]
 [  91  206  456  587  649 1137]
 [  91  206  456  587  649 1137]
 [  37  189  408  630  732  809]
 [  91  206  456  587  649 1137]
 [  91  206  456  587  649 1137]
 [  91  206  456  587  649 1137]
 [  91  206  456  587  649 1137]
 [  91  206  456  587  649 1137]
 [  91  206  456  587  649 1137]
 [ 189  226  408  630 1223 1233]
 [  91  206  456  587  649 1137]
 [  91  206  456  587  649 1137]
 [  91  206  456  587  649 1137]
 [  91  206  456  587  649 1137]
 [  91  206  456  587  649 1137]
 [  91  206  456  587  649 1137]
 [  91  206  456  587  649 1137]
 [  91  206  456  587  649 1137]
 [  91  206  456  587  649 1137]
 [  91  206  456  587  649 1137]
 [  37  189  408  630  732 1223]
 [  91  206  456  587  649 1137]
 [  91  206  456  587  649 1137]
 [  91  226  456  587 1223 1371]]


In [184]:
k

array([  37,   91,  189,  206,  226,  408,  456,  587,  630,  649,  732,
        809, 1137, 1223, 1233, 1371], dtype=int64)

In [185]:
title_encoded2title[302]

'Game Of Thrones Season 2 All Deleted Scenes'

# 2. Ranking

### Load dmodel and preprocess data

In [186]:
model = tf.keras.models.load_model(
    'candidate_generation.h5',
    custom_objects={
        'L2NormLayer': L2NormLayer,
        "MaskedEmbeddingsAggregatorLayer": MaskedEmbeddingsAggregatorLayer
    }
)

# custom_object: not available in tensorflow 

In [187]:
def get_genres(movies, genres):
    def get_all_genres(gs):
        active = [str(genres_encoded[genre]) for genre, g in zip(genres, gs) if g==1]
        if len(active) == 0:
            return '0'
        return ','.join((active))
    movies['category_term	'] = [
    get_all_genres(gs) for gs in zip(*[movies[genre] for genre in genres])]

In [188]:
title2title_encoded

{'Breastfeeding and your period': 0,
 'Lesson 1: Cloth Diaper Styles - Cloth Diapering 101': 1,
 'Kei To The City [Drift Feature Film - Japan]': 2,
 'MCM QLD Meet 2014': 3,
 'Game of Thrones - Lindsey Stirling & Peter Hollens (Cover)': 4,
 "Air France Flight 447's harrowing end": 5,
 'Sarah McLachlan - Possession [Official Music Video]': 6,
 'Jeff Buckley - Last Goodbye (Edit)': 7,
 'Jeff Buckley - So Real': 8,
 'Huey Lewis And The News - I Want A New Drug (Official Music Video)': 9,
 'Phil Wickham - This Is Amazing Grace (Official Music Video)': 10,
 'Are These Random Facts True or False?': 11,
 'Ingrid Michaelson - "Maybe" (Official Music Video)': 12,
 "'Breaking Bad's' Laura Fraser on the Remainder of Season 5": 13,
 'Red Hot Chili Peppers - Under The Bridge [Video]': 14,
 "'Pussy, get out of my pants' comment has tv hosts losing it": 15,
 'Citizenship Test': 16,
 'Borat speaking hebrew with israeli journalist': 17,
 'The Ultimate Fighter 17: Knockout of the Season': 18,
 'GLORY 16 

In [189]:
genres_encoded

{'Education': 0,
 'People & Blogs': 1,
 'Autos & Vehicles': 2,
 'Music': 3,
 'News & Politics': 4,
 'Entertainment': 5,
 'Comedy': 6,
 'Sports': 7,
 'Film & Animation': 8,
 'Pets & Animals': 9,
 'Howto & Style': 10,
 'Science & Technology': 11,
 'Travel & Events': 12,
 'Gaming': 13,
 'Shows': 14,
 'Nonprofits & Activism': 15}

In [190]:
videos


,video_id,video_title,viewCount,favoriteCount,dislikeCount,likeCount,duration,publishedAt,elapsed_time,category_term,...,engagement,general_engagement,discussion_density,like_percentage,dislike_percentage,like_dislike_ratio,share_percentage,example_age,like/dislike,year
0,0,Breastfeeding and your period,354581,0,169,435,294,2013-10-18T14:45:57.000Z,235,Education,...,0.337845,0.194216,0.058824,0.001227,0.000477,0.720199,0.000017,-0.547329,False,2013
1,1,Lesson 1: Cloth Diaper Styles - Cloth Diaperin...,37647,0,12,404,742,2012-04-13T06:25:23.000Z,798,People & Blogs,...,1.000000,0.000000,0.030303,0.010731,0.000319,0.971154,0.000717,0.306015,True,2012
2,2,Kei To The City [Drift Feature Film - Japan],853761,0,571,48772,3756,2014-02-26T22:00:01.000Z,82,Autos & Vehicles,...,0.070988,0.340177,0.046490,0.057126,0.000669,0.988428,0.001894,-0.749816,True,2014
3,3,MCM QLD Meet 2014,232819,0,68,6382,449,2014-04-03T22:21:04.000Z,46,Autos & Vehicles,...,0.037697,0.644937,0.150376,0.027412,0.000292,0.989457,0.000374,-0.805356,False,2014
4,4,Game of Thrones - Lindsey Stirling & Peter Hol...,4621920,0,1551,125076,196,2012-09-05T18:41:34.000Z,621,Music,...,0.882449,0.609950,0.130719,0.027061,0.000336,0.987751,0.004370,0.081615,True,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,1430,Dear 40...,111320,0,40,11280,176,2013-12-09T00:56:55.000Z,179,Comedy,...,1.000000,0.736436,0.036154,0.101330,0.000359,0.996466,0.000153,-0.626633,True,2013
1808,1431,"Toro Y Moi ""Talamak""",2035377,0,157,15173,161,2010-03-09T17:30:08.000Z,1550,Music,...,0.508373,0.000000,0.018367,0.007455,0.000077,0.989759,0.000000,1.486589,False,2010
1810,1432,Freegans: Creative Living Outside of Capitalism,65765,0,29,924,535,2012-10-17T16:58:04.000Z,584,Nonprofits & Activism,...,1.000000,0.527875,0.059322,0.014050,0.000441,0.969570,0.004166,0.016956,True,2012
1811,1433,Always That ONE THING...,73632,0,22,5230,203,2014-05-31T19:38:42.000Z,26,People & Blogs,...,1.000000,0.711601,0.061828,0.071029,0.000299,0.995811,0.000054,-0.894626,True,2014


In [191]:
video_data = videos.set_index(['video_id']).sort_index()
video_data = video_data.loc[k+1]
video_data



,video_title,viewCount,favoriteCount,dislikeCount,likeCount,duration,publishedAt,elapsed_time,category_term,watchTime,...,engagement,general_engagement,discussion_density,like_percentage,dislike_percentage,like_dislike_ratio,share_percentage,example_age,like/dislike,year
video_id,,,,,,,,,,,,,,,,,,,,,
38,James Harden Has a MEAN Crossover! NBA Kings o...,155550,0,37,2049,104,2014-03-04T19:42:52.000Z,90,Sports,-0.118034,...,1.000000,0.798697,0.263158,0.013173,0.000238,0.982263,0.000341,-0.758922,False,2014
92,Miley Cyrus - Wrecking Ball (Official Video),678405254,0,1218741,2903313,222,2013-09-09T16:00:38.000Z,292,Music,32.258568,...,1.000000,0.755715,0.018412,0.004280,0.001796,0.704336,0.001464,-0.487265,True,2013
190,How To Edit A Youtube Video (Easy),232592,0,370,291,154,2010-08-20T21:40:53.000Z,1385,Howto & Style,-0.121710,...,0.821805,0.000000,0.013575,0.001251,0.001591,0.440242,0.000000,1.233408,False,2010
207,ABM: Knuckles Vs Rouge Sonic Adventure 2 Battle,191409,0,252,3330,1962,2014-03-19T02:42:47.000Z,52,Entertainment,-0.101532,...,0.009869,0.188861,0.116402,0.017397,0.001317,0.929648,0.000047,-0.780962,False,2014
227,Montreal vs Toronto: which has a better dating...,29108,0,32,465,729,2013-12-19T13:30:15.000Z,172,Entertainment,-0.119128,...,0.186849,0.427783,0.465347,0.015975,0.001099,0.935614,0.001374,-0.642861,False,2013
409,Adelitas Way - Invincible,618252,0,88,8020,190,2011-01-28T01:58:24.000Z,1274,Music,-0.121710,...,1.000000,0.000000,0.054054,0.012972,0.000142,0.989147,0.005095,0.986389,True,2011
457,Godzilla Movie CLIP - Let Them Fight (2014) - ...,323943,0,384,10453,49,2014-05-03T04:58:56.000Z,1,Film & Animation,-0.117048,...,1.000000,1.032484,0.431818,0.032268,0.001185,0.964566,0.002806,-0.850504,False,2014
588,Makeup Tutorial - Naked Palette,223008,0,635,3560,718,2013-04-08T18:19:27.000Z,426,Howto & Style,-0.105162,...,0.088400,0.363267,0.018349,0.015964,0.002847,0.848629,0.000108,-0.249923,False,2013
631,Panic! At The Disco: Northern Downpour [OFFICI...,2830405,0,326,39468,251,2009-10-29T21:20:43.000Z,1703,Music,-0.121710,...,1.000000,0.000000,0.016245,0.013944,0.000115,0.991808,0.000000,1.688364,True,2009


In [192]:
video_data["genre_d"] = video_data["category_term"].map(genres_encoded)
video_data["title_d"] = video_data["video_title"].map(title2title_encoded)
video_data

,video_title,viewCount,favoriteCount,dislikeCount,likeCount,duration,publishedAt,elapsed_time,category_term,watchTime,...,discussion_density,like_percentage,dislike_percentage,like_dislike_ratio,share_percentage,example_age,like/dislike,year,genre_d,title_d
video_id,,,,,,,,,,,,,,,,,,,,,
38,James Harden Has a MEAN Crossover! NBA Kings o...,155550,0,37,2049,104,2014-03-04T19:42:52.000Z,90,Sports,-0.118034,...,0.263158,0.013173,0.000238,0.982263,0.000341,-0.758922,False,2014,7,38
92,Miley Cyrus - Wrecking Ball (Official Video),678405254,0,1218741,2903313,222,2013-09-09T16:00:38.000Z,292,Music,32.258568,...,0.018412,0.004280,0.001796,0.704336,0.001464,-0.487265,True,2013,3,92
190,How To Edit A Youtube Video (Easy),232592,0,370,291,154,2010-08-20T21:40:53.000Z,1385,Howto & Style,-0.121710,...,0.013575,0.001251,0.001591,0.440242,0.000000,1.233408,False,2010,10,190
207,ABM: Knuckles Vs Rouge Sonic Adventure 2 Battle,191409,0,252,3330,1962,2014-03-19T02:42:47.000Z,52,Entertainment,-0.101532,...,0.116402,0.017397,0.001317,0.929648,0.000047,-0.780962,False,2014,5,207
227,Montreal vs Toronto: which has a better dating...,29108,0,32,465,729,2013-12-19T13:30:15.000Z,172,Entertainment,-0.119128,...,0.465347,0.015975,0.001099,0.935614,0.001374,-0.642861,False,2013,5,227
409,Adelitas Way - Invincible,618252,0,88,8020,190,2011-01-28T01:58:24.000Z,1274,Music,-0.121710,...,0.054054,0.012972,0.000142,0.989147,0.005095,0.986389,True,2011,3,409
457,Godzilla Movie CLIP - Let Them Fight (2014) - ...,323943,0,384,10453,49,2014-05-03T04:58:56.000Z,1,Film & Animation,-0.117048,...,0.431818,0.032268,0.001185,0.964566,0.002806,-0.850504,False,2014,8,457
588,Makeup Tutorial - Naked Palette,223008,0,635,3560,718,2013-04-08T18:19:27.000Z,426,Howto & Style,-0.105162,...,0.018349,0.015964,0.002847,0.848629,0.000108,-0.249923,False,2013,10,588
631,Panic! At The Disco: Northern Downpour [OFFICI...,2830405,0,326,39468,251,2009-10-29T21:20:43.000Z,1703,Music,-0.121710,...,0.016245,0.013944,0.000115,0.991808,0.000000,1.688364,True,2009,3,631


In [193]:
#video_data['video_id']=video_data['title_d']
new_data = video_data.merge(df[['user_id','video_id']], on='video_id') # rating 추가

#video_data = video_data.reindex(columns=[ 'video_id', 'user_id','watchTime', 'genre_d', 'title_d','like/dislike'])
new_data= new_data[[ 'video_id', 'user_id','watchTime', 'genre_d', 'title_d','like/dislike']]
new_data['like/dislike'] = new_data['like/dislike'].map(likeMap)
new_data

,video_id,user_id,watchTime,genre_d,title_d,like/dislike
0,38,9,-0.118034,7,38,dislike
1,92,20,32.258568,3,92,like
2,190,27,-0.121710,10,190,dislike
3,207,27,-0.101532,5,207,dislike
4,227,32,-0.119128,5,227,dislike
5,409,58,-0.121710,3,409,like
6,457,67,-0.117048,8,457,dislike
7,588,79,-0.105162,10,588,dislike
8,631,84,-0.121710,3,631,like
9,650,84,-0.117309,5,650,like


In [194]:
video_list = new_data.groupby(['user_id','like/dislike'])['video_id'].apply(list).reset_index()

video_list

,user_id,like/dislike,video_id
0,9,dislike,[38]
1,20,like,[92]
2,27,dislike,"[190, 207]"
3,32,dislike,[227]
4,58,like,[409]
5,67,dislike,[457]
6,79,dislike,[588]
7,84,like,"[631, 650]"
8,92,dislike,[733]
9,101,like,[810]


In [195]:

genre_list = new_data.groupby(['user_id'])['genre_d'].unique().apply(list).reset_index()
#genre_list['genre_d']=genre_list['genre_d'].apply(lambda x: list(set(','.join(x))) ) # 중복제거
#genre_list['genre_d']=genre_list['genre_d'].apply(lambda x:[ x for x in x if x.isdigit() ])

new_data = normalize_col(new_data, 'watchTime')
timestamp_list = new_data.groupby(['user_id'])['watchTime'].unique().apply(list).reset_index()

title_list = new_data.groupby(['user_id'])['title_d'].apply(list).reset_index()
print(video_list)
dataset = video_list.pivot(index='user_id', columns='like/dislike', values='video_id').reset_index()
dataset.fillna(new_data["video_id"].max()+1, inplace=True)

dataset['like'] =dataset['like'].apply(lambda x: x if type(x) is list else [])
dataset['dislike'] =dataset['dislike'].apply(lambda x: x if type(x) is list else [])

dataset = pd.merge(dataset, title_list, how='left')
dataset = pd.merge(dataset, genre_list, how='left')
dataset = pd.merge(dataset, timestamp_list, how='left')

dataset['predict_labels'] = dataset['like'].apply(lambda x: int(random.uniform(1,new_data["video_id"].max()))) #label을 마지막 값으로..

dataset['like']=dataset['like'].apply(lambda x: [new_data["video_id"].max()+1] if x == [] else x)
dataset['dislike']=dataset['dislike'].apply(lambda x: [new_data["video_id"].max()+1] if x == [] else x)
#train_data=dataset[(dataset.user_id >= 1)& (dataset.user_id <= 5)]
#test_data=dataset[(dataset.user_id >= 6)&(dataset.user_id <= 9)]
#dataset=dataset.drop(columns=['video_id'])

    user_id like/dislike      video_id
0         9      dislike          [38]
1        20         like          [92]
2        27      dislike    [190, 207]
3        32      dislike         [227]
4        58         like         [409]
5        67      dislike         [457]
6        79      dislike         [588]
7        84         like    [631, 650]
8        92      dislike         [733]
9       101         like         [810]
10      125      dislike        [1138]
11      134      dislike  [1224, 1234]
12      141      dislike        [1372]


In [196]:
# split test and train set
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(dataset, test_size=0.2)

In [197]:
train_data

,user_id,dislike,like,title_d,genre_d,watchTime,predict_labels
6,79,[588],[1373],[588],[10],[-0.2488327337077448],694
4,58,[1373],[409],[409],[3],[-0.2508774151742591],539
3,32,[227],[1373],[227],[5],[-0.25055832182676585],344
8,92,[733],[1373],[733],[11],[-0.2454248767193101],1195
11,134,"[1224, 1234]",[1373],"[1224, 1234]","[4, 3]","[-0.2502410635381971, -0.2508774151742591]",721
2,27,"[190, 207]",[1373],"[190, 207]","[10, 5]","[-0.2508774151742591, -0.24838420582105183]",818
12,141,[1372],[1373],[1372],[8],[-0.25023241876221836],528
1,20,[1373],[92],[92],[3],[3.749996204212265],869
5,67,[457],[1373],[457],[8],[-0.25030130872344286],758
0,9,[38],[1373],[38],[7],[-0.25042322286992696],876


In [198]:
EMBEDDING_DIMS = 16
DENSE_UNITS = 64
DROPOUT_PCT = 0.0
ALPHA = 0.0
NUM_CLASSES=df["video_id"].max() + 3
LEARNING_RATE = 0.0003

In [199]:
#---inputs
import tensorflow as tf
import datetime
import os
input_title = tf.keras.Input(shape=(None, ), name='video_title')
inp_video_liked = tf.keras.layers.Input(shape=(None,), name='like')
inp_video_disliked = tf.keras.layers.Input(shape=(None,), name='dislike')
input_genre = tf.keras.Input(shape=(None, ), name='genre')
input_timestamp = tf.keras.Input(shape=(None, ), name='timestamp')
# input_occupation = tf.keras.Input(shape=(None, ), name='occupation')


#--- layers
features_embedding_layer = tf.keras.layers.Embedding(input_dim=NUM_CLASSES, output_dim=EMBEDDING_DIMS, 
                                            mask_zero=True, trainable=True, name='features_embeddings')
labels_embedding_layer = tf.keras.layers.Embedding(input_dim=NUM_CLASSES, output_dim=EMBEDDING_DIMS, 
                                            mask_zero=True, trainable=True, name='labels_embeddings')

avg_embeddings = MaskedEmbeddingsAggregatorLayer(agg_mode='mean', name='aggregate_embeddings')

dense_1 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_1')
dense_2 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_2')
dense_3 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_3')
l2_norm_1 = L2NormLayer(name='l2_norm_1')

dense_output = tf.keras.layers.Dense(NUM_CLASSES, activation=tf.nn.softmax, name='dense_output')

#--- features
features_embeddings = features_embedding_layer(input_title)
l2_norm_features = l2_norm_1(features_embeddings)
avg_features = avg_embeddings(l2_norm_features)

labels_liked_embeddings = labels_embedding_layer(inp_video_liked)
l2_norm_liked = l2_norm_1(labels_liked_embeddings)
avg_liked = avg_embeddings(l2_norm_liked)

labels_disliked_embeddings = labels_embedding_layer(inp_video_disliked)
l2_norm_disliked = l2_norm_1(labels_disliked_embeddings)
avg_disliked = avg_embeddings(l2_norm_disliked)

labels_genre_embeddings = labels_embedding_layer(input_genre)
l2_norm_genre = l2_norm_1(labels_genre_embeddings)
avg_genre = avg_embeddings(l2_norm_genre)

labels_timestamp_embeddings = labels_embedding_layer(input_timestamp)
l2_norm_timestamp = l2_norm_1(labels_timestamp_embeddings)
avg_timestamp = avg_embeddings(l2_norm_timestamp)


# 임베딩 벡터들 연결
concat_inputs = tf.keras.layers.Concatenate(axis=1)([avg_features,
                                                     avg_liked,
                                                     avg_disliked,
                                                     avg_genre,
                                                     avg_timestamp
                                                     ])
# Dense Layers
dense_1_features = dense_1(concat_inputs)
dense_1_relu = tf.keras.layers.ReLU(name='dense_1_relu')(dense_1_features)
dense_1_batch_norm = tf.keras.layers.BatchNormalization(name='dense_1_batch_norm')(dense_1_relu)

dense_2_features = dense_2(dense_1_relu)
dense_2_relu = tf.keras.layers.ReLU(name='dense_2_relu')(dense_2_features)
# dense_2_batch_norm = tf.keras.layers.BatchNormalization(name='dense_2_batch_norm')(dense_2_relu)

dense_3_features = dense_3(dense_2_relu)
dense_3_relu = tf.keras.layers.ReLU(name='dense_3_relu')(dense_3_features)
dense_3_batch_norm = tf.keras.layers.BatchNormalization(name='dense_3_batch_norm')(dense_3_relu)
outputs = dense_output(dense_3_batch_norm)

#Optimizer
optimiser = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

#--- prep model
model = tf.keras.models.Model(
    inputs=[input_title, 
            inp_video_liked, 
            inp_video_disliked,
            input_genre,
            input_timestamp,
            ],
    outputs=[outputs]
)
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
model.compile(optimizer=optimiser, loss='sparse_categorical_crossentropy', metrics=['acc'])

 #model.summary()

In [200]:
#train_data= train_data.fillna(0)
train_data

,user_id,dislike,like,title_d,genre_d,watchTime,predict_labels
6,79,[588],[1373],[588],[10],[-0.2488327337077448],694
4,58,[1373],[409],[409],[3],[-0.2508774151742591],539
3,32,[227],[1373],[227],[5],[-0.25055832182676585],344
8,92,[733],[1373],[733],[11],[-0.2454248767193101],1195
11,134,"[1224, 1234]",[1373],"[1224, 1234]","[4, 3]","[-0.2502410635381971, -0.2508774151742591]",721
2,27,"[190, 207]",[1373],"[190, 207]","[10, 5]","[-0.2508774151742591, -0.24838420582105183]",818
12,141,[1372],[1373],[1372],[8],[-0.25023241876221836],528
1,20,[1373],[92],[92],[3],[3.749996204212265],869
5,67,[457],[1373],[457],[8],[-0.25030130872344286],758
0,9,[38],[1373],[38],[7],[-0.25042322286992696],876


In [201]:
history = model.fit([tf.keras.preprocessing.sequence.pad_sequences(train_data['title_d'])+ 1e-10,
           tf.keras.preprocessing.sequence.pad_sequences(train_data['like']) + 1e-10,
           tf.keras.preprocessing.sequence.pad_sequences(train_data['dislike']) + 1e-10,
            tf.keras.preprocessing.sequence.pad_sequences(train_data['genre_d']) + 1e-10,
            tf.keras.preprocessing.sequence.pad_sequences(train_data['watchTime'], dtype=float) + 1e-5,
           ],train_data['predict_labels'].values,
           steps_per_epoch=1, epochs=500)

Epoch 1/500
1/1 [==============================] - 2s 2s/step - loss: 7.3387 - acc: 0.0000e+00
Epoch 2/500
1/1 [==============================] - 0s 4ms/step - loss: 7.2836 - acc: 0.0000e+00
Epoch 3/500
1/1 [==============================] - 0s 3ms/step - loss: 7.2326 - acc: 0.0000e+00
Epoch 4/500
1/1 [==============================] - 0s 5ms/step - loss: 7.1850 - acc: 0.0000e+00
Epoch 5/500
1/1 [==============================] - 0s 3ms/step - loss: 7.1401 - acc: 0.0000e+00
Epoch 6/500
1/1 [==============================] - 0s 3ms/step - loss: 7.0970 - acc: 0.0000e+00
Epoch 7/500
1/1 [==============================] - 0s 3ms/step - loss: 7.0558 - acc: 0.0000e+00
Epoch 8/500
1/1 [==============================] - 0s 4ms/step - loss: 7.0157 - acc: 0.1000
Epoch 9/500
1/1 [==============================] - 0s 4ms/step - loss: 6.9771 - acc: 0.1000
Epoch 10/500
1/1 [==============================] - 0s 3ms/step - loss: 6.9390 - acc: 0.1000
Epoch 11/500
1/1 [==============================] - 

1/1 [==============================] - 0s 3ms/step - loss: 2.9296 - acc: 1.0000
Epoch 176/500
1/1 [==============================] - 0s 3ms/step - loss: 2.9048 - acc: 1.0000
Epoch 177/500
1/1 [==============================] - 0s 4ms/step - loss: 2.8802 - acc: 1.0000
Epoch 178/500
1/1 [==============================] - 0s 3ms/step - loss: 2.8555 - acc: 1.0000
Epoch 179/500
1/1 [==============================] - 0s 3ms/step - loss: 2.8309 - acc: 1.0000
Epoch 180/500
1/1 [==============================] - 0s 3ms/step - loss: 2.8063 - acc: 1.0000
Epoch 181/500
1/1 [==============================] - 0s 4ms/step - loss: 2.7817 - acc: 1.0000
Epoch 182/500
1/1 [==============================] - 0s 3ms/step - loss: 2.7573 - acc: 1.0000
Epoch 183/500
1/1 [==============================] - 0s 3ms/step - loss: 2.7328 - acc: 1.0000
Epoch 184/500
1/1 [==============================] - 0s 3ms/step - loss: 2.7082 - acc: 1.0000
Epoch 185/500
1/1 [==============================] - 0s 4ms/step - loss: 2

1/1 [==============================] - 0s 3ms/step - loss: 0.3037 - acc: 1.0000
Epoch 350/500
1/1 [==============================] - 0s 4ms/step - loss: 0.3002 - acc: 1.0000
Epoch 351/500
1/1 [==============================] - 0s 3ms/step - loss: 0.2967 - acc: 1.0000
Epoch 352/500
1/1 [==============================] - 0s 3ms/step - loss: 0.2933 - acc: 1.0000
Epoch 353/500
1/1 [==============================] - 0s 3ms/step - loss: 0.2900 - acc: 1.0000
Epoch 354/500
1/1 [==============================] - 0s 4ms/step - loss: 0.2867 - acc: 1.0000
Epoch 355/500
1/1 [==============================] - 0s 3ms/step - loss: 0.2835 - acc: 1.0000
Epoch 356/500
1/1 [==============================] - 0s 3ms/step - loss: 0.2803 - acc: 1.0000
Epoch 357/500
1/1 [==============================] - 0s 3ms/step - loss: 0.2772 - acc: 1.0000
Epoch 358/500
1/1 [==============================] - 0s 4ms/step - loss: 0.2742 - acc: 1.0000
Epoch 359/500
1/1 [==============================] - 0s 3ms/step - loss: 0